In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from gsheet import *
from mailer import *
from qr_code import *
from payment_data import *
import pandas as pd
import logging

Succesfully read config from: ['./config_prispevky.cfg']


In [5]:
logging.basicConfig(
    format="[%(levelname)s] %(message)s",
    level=logging.INFO
)

In [6]:
sheet = Sheets(SPREADSHEET_ID)

[INFO] file_cache is only supported with oauth2client<4.0.0


In [7]:
# sheet_names = sheet.get_sheet_names()
# sheet_name = '4pvs_all'
# sheet_name = 'PLATBY_PODZIM_2022_filtered'
sheet_name = 'TESTv2'
if 'TEST' in sheet_name:
    testmode=True
else:
    testmode=False
df = sheet.get_dataframe(sheet_name)

In [8]:
df

,Osoba,Evidenční číslo,Jednotka,Typ jednotky,Typ členství,Kategorie,Registrační číslo,E-mail (další),E-mail (hlavní),Mobil (další),...,Matka: mail,Matka: telefon,Ostatní: mail,Ostatní: telefon,2/2021,Celkem Poplatek 2022,Poplatek 2/2022,STS,Zaplaceno,Zaplaceno STS
0,"Unpaid prispevky, NO sts",113.04.004-1,Albatrosové,Oddíl,Řádné,Skaut,10049,,mrkosjan@seznam.cz,,...,mrkosjan@gmail.com,,,,,1400,700,0,0,None
1,"Unpaid prispevky, paid sts",113.04.004-1,Kačky,Oddíl,Řádné,Skaut,20148,,,,...,mrkosjan@gmail.com,606635345,,,,500,500,400,0,400
2,"Paid prispevky, unpaid sts",113.04.004-1,Bobři,Oddíl,Řádné,Skaut,14811,,,,...,mrkosjan@gmail.com,606635345,,,,1400,700,200,700,None
3,"Unapid prispecky, unpaid STS",113.04.004-1,Albatrosové,Oddíl,Řádné,Skaut,10126,,,,...,"mrkosjan@gmail.com, hondzik@skaut.cz",737617645,,,700,1400,700,200,0,None
4,All paid - should not send,,Albatrosové,,,,12345,mrkosja1@fel.cvut.cz,,,...,,,,,700,1400,700,200,700,200
5,No payment - should not send,113.04.004-1,Rackové,Oddíl,Řádné,Skaut,10038,,,,...,"mrkosjan@gmail.com, mrkosja1@fel.cvut.cz",739390266,,,,0,0,0,0,None
6,Invalid email - should not send,,Albatrosové,,,,476456,,,,...,,,dsadssadasga.cz,,,0,0,0,0,0
7,No email - should not send,113.04.004-1,Albatrosové,Oddíl,Řádné,Skaut,10039,,,,...,,739390266,None,None,None,None,None,None,None,None


In [9]:
NEEDED_COLS = NeededColumns()
pdf=PaymentsDataFrame(df, NEEDED_COLS)

[WARNING] Incorrect email address read: 'Ostatní: mail': 'dsadssadasga.cz'; addr-spec local part with no domain
[INFO] Of 8 people, missing email: 2,
	emailable paid: 3
	emailable unpaid: 3.
 Also there were 1 invalid emails.


In [2]:
pdf.df_missing_email

NameError: name 'pdf' is not defined

In [ ]:
# !!!NOTE: READ FROM CONFIG FILE!!!
# SENDER_ADDRESS = Address(CONFIG['mailer']['sender_name'], addr_spec=CONFIG['mailer']['sender_address'])
# EMAIL_SUBJECT = CONFIG['mailer']['subject']

In [ ]:
emails = []

"""
!!!!!WARNING! CURRENTLY SET TO EMAIL ALL AVAILABLE EMAIL ADDRESSES!!!!!
"""

for i, row in pdf.df_emailable_unpaid.iterrows():
    recepient_addresses = row.valid_addresses
    pi = PaymentInfo.from_df_row(row, NEEDED_COLS)
    email = PaymentEmail(pi, recepient_addresses, testmode=testmode)
    emails.append(email.msg)

In [ ]:
with open(f'./test.eml', 'wb') as f:
    f.write(bytes(emails[0]))

In [ ]:
mailer = Mailer()
sent = []

In [ ]:
for email in emails[1:]:
    sent_msg = mailer.send_message(str(email))
    sent.append((email, sent_msg))